In [2]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import math
import os
import itertools
import warnings
import glob
import sys
staff = pd.read_csv("people_flt.csv")
staff_flt = staff[(~staff["MTMT"].isnull()) & (staff["MTMT"]!=11111111) & (staff["MTMT"]!=0) & (staff["Munkakör"].str.lower()!="doktorandusz") & (staff["Munkakör"].str.lower()!="sh doktorandusz")]
staff_flt["MTMT"] = staff_flt["MTMT"].astype("int")

/tmp/ipykernel_364067/3833529470.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staff_flt["MTMT"] = staff_flt["MTMT"].astype("int")


In [ ]:
def load_pub_for_author(mtid):
    page = 1
    size = 1000
    tries = 1
    author_pubs = []
    while True:
        os.system(f'wget -O tmp.json "https://m2.mtmt.hu/api/publication?cond=published;eq;true&cond=core;eq;true&cond=authors.mtid;eq;{mtid}&sort=publishedYear,desc&sort=firstAuthor,asc&size={size}&fields=citations,pubStats&labelLang=hun&page={page}&format=json" >wget.out 2>wget.err')
        with open("tmp.json", "r") as f:
            try:
                dta = json.load(f)
                if page == 1 and len(dta["content"]) == 0:
                    return []
                author_pubs += dta["content"]
                if not dta["paging"]["last"]:
                    page += 1
                else:
                    break
            except:
                tries += 1
                print("Error, retrying")
                if tries > 30:
                    return []
    return author_pubs

In [6]:
for mtid in tqdm(staff_flt["MTMT"]):
    if not os.path.exists(f"3_1/{mtid}.json"):
        json.dump(load_pub_for_author(mtid), open(f"3_1/{mtid}.json", "w", encoding='utf8'), indent=4, ensure_ascii=False)

100%|█████████████████████████████████████| 119/119 [00:00<00:00, 131037.59it/s]


In [7]:
ifdf = pd.read_csv('./pubgraph/ifdf_v8_2021.csv')
ifdf.loc[ifdf["if"].isnull(), "if"] = 0.0
maxIFYear = ifdf.year.max()
ifdf_e = ifdf.copy().set_index(["year", "eissn"]).sort_index()
ifdf_p = ifdf.copy().set_index(["year", "pissn"]).sort_index()

def getif(pub):
    if "journal" not in pub:
        return 0.0, 1.0
    year = min(pub["publishedYear"], maxIFYear)
    if "pIssn" in pub["journal"]:
        issn = pub["journal"]["pIssn"]
        if (year,issn) in ifdf_p.index:
            ifval = ifdf_p.loc[(year,issn),"if"]
            catif = ifdf_p.loc[(year,issn),"categoryMedianIf"]
            if len(ifval) == 1 and float(ifval) > 0.0:
                if len(catif) == 1 and float(catif) > 0.0:
                    return float(ifval), float(catif)
                else:
                    return float(ifval), float(ifval) # ha if-es az újság, de még nem elég régóta. nincs sok ilyen.
    if "eIssn" in pub["journal"]:
        issn = pub["journal"]["eIssn"]
        if (year,issn) in ifdf_e.index:
            ifval = ifdf_e.loc[(year,issn),"if"]
            catif = ifdf_e.loc[(year,issn),"categoryMedianIf"]
            if len(ifval) == 1 and float(ifval) > 0.0:
                if len(catif) == 1 and float(catif) > 0.0:
                    return float(ifval), float(catif)
                else:
                    return float(ifval), float(ifval) # ha if-es az újság, de még nem elég régóta. nincs sok ilyen.
    return 0.0, 1.0

# a függvény visszaadja a folyóirat rangját (D1, Q1, Q2, ...)
def getrating(pub):
    if "ratings" not in pub:
        return ""
    for r in pub["ratings"]:
        if r["otype"] == "SjrRating" and "ranking" in r:
            return r["ranking"]
    return ""
def score_person(mtid):
    if not os.path.exists(f'3_1/{mtid}.json'):
        raise Exception("Nincs letöltve a publikációs lista!")
    pubs = json.load(open(f'3_1/{mtid}.json'))
    score = score_publist([mtid], pubs)
    score["mtid"] = int(mtid)
    return score
    
def score_publist(mtid_list, pubs):
    score = {"pub_count": 0, "q_paper": 0, "q_n_paper": 0, "q_book": 0, "if": 0, "relif": 0, "ifcnt": 0, "if_norm": 0, "relif_norm": 0, "h": 0, "i": 0, "first_pub": 2100, "last_pub": 0}
    cit_list = []
    useShare = False
    ix = 0
    for pub in pubs:
        ix += 1
        if "error" in pub and pub["error"]!="VALIDATION_ERROR":
            continue
        if "category" not in pub or pub["category"]["label"]!="Tudományos":
            continue
        # compute share of the authos
        authors = 0
        share = len(mtid_list)/len(pub["authorships"]) if len(pub["authorships"])>0 else 0
        for a in pub["authorships"]:
            if "author" in a and a["author"]["mtid"] in mtid_list and a["authorTyped"]:
                authors += 1
        if authors < len(mtid_list):
            continue
        # compute number of pages
        plength = 0
        try:
            if "pageLength" in pub:
                plength = int(pub["pageLength"])
            elif "firstPage" in pub and "lastPage" in pub:
                plength = int(pub["lastPage"]) - int(pub["firstPage"]) + 1            
        except:
            pass
        # compute impact factor
        ifct, nrm = getif(pub)        
        # compute Q score
        if pub["otype"] == "JournalArticle" and "journal" in pub:
            if "fullPublication" in pub and pub["fullPublication"] and "reviewType" in pub["journal"] and pub["journal"]["reviewType"]=="REVIEWED" and "subType" in pub and \
                pub["subType"]["name"] in ["Szakcikk", "Összefoglaló cikk", "Konferenciaközlemény", "Rövid közlemény", "Sokszerzős vagy csoportos szerzőségű szakcikk"]:                
                if ifct > 0:
                    totalscr = max(0.6,ifct)
                elif "foreignEdition" in pub and pub["foreignEdition"]:
                    totalscr = 0.4
                else:
                    totalscr = 0.3
                score["q_paper"] += totalscr * share
                score["q_n_paper"] += totalscr * share / nrm if nrm > 0 else totalscr * share
                if ifct > 0:
                    score["if"] += ifct
                    score["relif"] += totalscr * share
                    score["ifcnt"] += 1
                    score["if_norm"] += ifct / nrm if nrm > 0 else ifct
                    score["relif_norm"] += totalscr * share / nrm if nrm > 0 else totalscr * share
        elif "fullPublication" in pub and pub["fullPublication"] and \
            ((pub["type"]["label"]=="Könyvrészlet" and "subType" in pub and pub["subType"]["label"]=="Konferenciaközlemény (Könyvrészlet)") or pub["type"]["label"]=="Egyéb konferenciaközlemény"):                
            if plength >= 4:
                totalscr = 0.2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 0.1
                score["q_paper"] += totalscr * share
                score["q_n_paper"] += totalscr * share
        elif pub["type"]["label"] == "Könyv" or \
            (pub["type"]["label"] == "Könyvrészlet" and "subType" in pub and pub["subType"]["label"] in ["Könyvfejezet (Könyvrészlet)", "Szaktanulmány (Könyvrészlet)"]):
            if plength >= 10:
                if plength >= 100:
                    totalscr = 2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 1
                else:
                    totalscr = (0.2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 0.1) * math.floor(plength/10)
                score["q_book"] += totalscr * share
        # Process citations
        if "independentCitationCount" in pub:
            score["i"] += pub["independentCitationCount"]
            cit_list.append(pub["independentCitationCount"])
        if "publishedYear" in pub:
            score["last_pub"] = max(score["last_pub"], pub["publishedYear"])
            score["first_pub"] = min(score["first_pub"], pub["publishedYear"])
        score["pub_count"] += 1
    score["q"] = score["q_paper"] + np.minimum(score["q_book"], 3.0)
    score["qn"] = score["q_n_paper"] + np.minimum(score["q_book"], 3.0)
    # h-index
    cit_list.sort(reverse=True)
    while score["h"] < len(cit_list) and score["h"] < cit_list[score["h"]]:
        score["h"] += 1
        
    return score
def get_h_score(mtid):
    cit_list = []
    if os.path.exists(f'3_1/{mtid}.json'):
        with open(f'3_1/{mtid}.json') as f:
            try:
                js = json.load(f)
                for pub in js:
                    if "independentCitationCount" in pub:
                        cit_list.append(pub["independentCitationCount"])
            except:
                pass
    cit_list.sort(reverse=True)
    hindex = 0
    while hindex < len(cit_list) and hindex < cit_list[hindex]:
        hindex += 1
    return hindex
staff_flt["H index"] = [0 if np.isnan(mtid) else get_h_score(int(mtid)) for mtid in staff_flt["MTMT"]]
dta = {"MTMT": [], "pubCount": [], "qScore": [], "qnScore": [], "ifCount": [], "ifScore": [], "relifScore": [], "ifNormScore": [], "relifNormScore": [], "citations": [], "hIndex": [], "firstPub": [], "lastPub": []}
for idx, prs in staff_flt.iterrows():
    scr = score_person(prs["MTMT"])
    dta["MTMT"].append(scr["mtid"])
    dta["pubCount"].append(scr["pub_count"])
    dta["qScore"].append(scr["q"])
    dta["qnScore"].append(scr["qn"])
    dta["ifCount"].append(scr["ifcnt"])
    dta["ifScore"].append(scr["if"])
    dta["relifScore"].append(scr["relif"])
    dta["ifNormScore"].append(scr["if_norm"])
    dta["relifNormScore"].append(scr["relif_norm"])
    dta["citations"].append(scr["i"])
    dta["hIndex"].append(scr["h"])
    dta["firstPub"].append(scr["first_pub"])
    dta["lastPub"].append(scr["last_pub"])
node_person = pd.DataFrame(dta)
pd.set_option('display.max_rows', 15)
node_person.to_csv("people_no affiliation.csv", index=False, float_format='%.4g')
node_person

/tmp/ipykernel_1952561/4126662895.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staff_flt["H index"] = [0 if np.isnan(mtid) else get_h_score(int(mtid)) for mtid in staff_flt["MTMT"]]


,MTMT,pubCount,qScore,qnScore,ifCount,ifScore,relifScore,ifNormScore,relifNormScore,citations,hIndex,firstPub,lastPub
0,10012522,23,9.162250,6.800885,10,10.856,8.162250,7.796477,5.800885,98,7,1998,2020
1,10041405,36,7.200000,16.093861,6,0.918,2.400000,4.281639,11.293861,42,3,1970,2009
2,10011729,33,15.860783,12.435711,26,39.425,15.727450,28.957170,12.302377,219,8,1997,2021
3,10042381,15,1.733333,1.733333,0,0.000,0.000000,0.000000,0.000000,3,1,2008,2017
4,10011669,31,7.139625,3.533433,10,21.269,5.916898,8.404271,2.310706,168,8,1985,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,10019824,20,8.937917,7.132186,15,15.574,8.287917,12.140872,6.482186,200,6,2006,2022
115,10019824,20,8.937917,7.132186,15,15.574,8.287917,12.140872,6.482186,200,6,2006,2022
116,10042219,39,29.287500,14.404563,30,67.256,29.220833,31.425362,14.337897,144,7,2008,2023
117,10015273,33,25.605867,15.803946,30,53.344,25.372533,33.254772,15.570612,328,10,2001,2022


# affiliation

In [8]:
for mtid in tqdm(staff_flt["MTMT"]):
    if not os.path.exists(f"3_3/{mtid}.json"):
        json.dump(load_pub_for_author(mtid), open(f"3_3/{mtid}.json", "w", encoding='utf8'), indent=4, ensure_ascii=False)
def score_person(mtid):
    if not os.path.exists(f'3_3/{mtid}.json'):
        raise Exception("Nincs letöltve a publikációs lista!")
    pubs = json.load(open(f'3_3/{mtid}.json'))
    score = score_publist([mtid], pubs)
    score["mtid"] = int(mtid)
    return score
    
def score_publist(mtid_list, pubs):
    score = {"pub_count": 0, "q_paper": 0, "q_n_paper": 0, "q_book": 0, "if": 0, "relif": 0, "ifcnt": 0, "if_norm": 0, "relif_norm": 0, "h": 0, "i": 0, "first_pub": 2100, "last_pub": 0}
    cit_list = []
    useShare = False
    ix = 0
    for pub in pubs:
        ix += 1
        if "error" in pub and pub["error"]!="VALIDATION_ERROR":
            continue
        if "category" not in pub or pub["category"]["label"]!="Tudományos":
            continue
        # compute share of the authos
        authors = 0
        share = len(mtid_list)/len(pub["authorships"]) if len(pub["authorships"])>0 else 0
        for a in pub["authorships"]:
            if "author" in a and a["author"]["mtid"] in mtid_list and a["authorTyped"]:
                authors += 1
        if authors < len(mtid_list):
            continue
        # compute number of pages
        plength = 0
        try:
            if "pageLength" in pub:
                plength = int(pub["pageLength"])
            elif "firstPage" in pub and "lastPage" in pub:
                plength = int(pub["lastPage"]) - int(pub["firstPage"]) + 1            
        except:
            pass
        # compute impact factor
        ifct, nrm = getif(pub)        
        # compute Q score
        if pub["otype"] == "JournalArticle" and "journal" in pub:
            if "fullPublication" in pub and pub["fullPublication"] and "reviewType" in pub["journal"] and pub["journal"]["reviewType"]=="REVIEWED" and "subType" in pub and \
                pub["subType"]["name"] in ["Szakcikk", "Összefoglaló cikk", "Konferenciaközlemény", "Rövid közlemény", "Sokszerzős vagy csoportos szerzőségű szakcikk"]:                
                if ifct > 0:
                    totalscr = max(0.6,ifct)
                elif "foreignEdition" in pub and pub["foreignEdition"]:
                    totalscr = 0.4
                else:
                    totalscr = 0.3
                score["q_paper"] += totalscr * share
                score["q_n_paper"] += totalscr * share / nrm if nrm > 0 else totalscr * share
                if ifct > 0:
                    score["if"] += ifct
                    score["relif"] += totalscr * share
                    score["ifcnt"] += 1
                    score["if_norm"] += ifct / nrm if nrm > 0 else ifct
                    score["relif_norm"] += totalscr * share / nrm if nrm > 0 else totalscr * share
        elif "fullPublication" in pub and pub["fullPublication"] and \
            ((pub["type"]["label"]=="Könyvrészlet" and "subType" in pub and pub["subType"]["label"]=="Konferenciaközlemény (Könyvrészlet)") or pub["type"]["label"]=="Egyéb konferenciaközlemény"):                
            if plength >= 4:
                totalscr = 0.2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 0.1
                score["q_paper"] += totalscr * share
                score["q_n_paper"] += totalscr * share
        elif pub["type"]["label"] == "Könyv" or \
            (pub["type"]["label"] == "Könyvrészlet" and "subType" in pub and pub["subType"]["label"] in ["Könyvfejezet (Könyvrészlet)", "Szaktanulmány (Könyvrészlet)"]):
            if plength >= 10:
                if plength >= 100:
                    totalscr = 2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 1
                else:
                    totalscr = (0.2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 0.1) * math.floor(plength/10)
                score["q_book"] += totalscr * share
        # Process citations
        if "independentCitationCount" in pub:
            score["i"] += pub["independentCitationCount"]
            cit_list.append(pub["independentCitationCount"])
        if "publishedYear" in pub:
            score["last_pub"] = max(score["last_pub"], pub["publishedYear"])
            score["first_pub"] = min(score["first_pub"], pub["publishedYear"])
        score["pub_count"] += 1
    score["q"] = score["q_paper"] + np.minimum(score["q_book"], 3.0)
    score["qn"] = score["q_n_paper"] + np.minimum(score["q_book"], 3.0)
    # h-index
    cit_list.sort(reverse=True)
    while score["h"] < len(cit_list) and score["h"] < cit_list[score["h"]]:
        score["h"] += 1
        
    return score
def get_h_score(mtid):
    cit_list = []
    if os.path.exists(f'3_3/{mtid}.json'):
        with open(f'3_3/{mtid}.json') as f:
            try:
                js = json.load(f)
                for pub in js:
                    if "independentCitationCount" in pub:
                        cit_list.append(pub["independentCitationCount"])
            except:
                pass
    cit_list.sort(reverse=True)
    hindex = 0
    while hindex < len(cit_list) and hindex < cit_list[hindex]:
        hindex += 1
    return hindex
staff_flt["H index"] = [0 if np.isnan(mtid) else get_h_score(int(mtid)) for mtid in staff_flt["MTMT"]]
dta = {"MTMT": [], "pubCount": [], "qScore": [], "qnScore": [], "ifCount": [], "ifScore": [], "relifScore": [], "ifNormScore": [], "relifNormScore": [], "citations": [], "hIndex": [], "firstPub": [], "lastPub": []}
for idx, prs in staff_flt.iterrows():
    scr = score_person(prs["MTMT"])
    dta["MTMT"].append(scr["mtid"])
    dta["pubCount"].append(scr["pub_count"])
    dta["qScore"].append(scr["q"])
    dta["qnScore"].append(scr["qn"])
    dta["ifCount"].append(scr["ifcnt"])
    dta["ifScore"].append(scr["if"])
    dta["relifScore"].append(scr["relif"])
    dta["ifNormScore"].append(scr["if_norm"])
    dta["relifNormScore"].append(scr["relif_norm"])
    dta["citations"].append(scr["i"])
    dta["hIndex"].append(scr["h"])
    dta["firstPub"].append(scr["first_pub"])
    dta["lastPub"].append(scr["last_pub"])
node_person = pd.DataFrame(dta)
node_person.to_csv("people_affiliation.csv", index=False, float_format='%.4g')
node_person

100%|██████████████████████████████████████| 119/119 [00:00<00:00, 87030.89it/s]
/tmp/ipykernel_1952561/3405208838.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staff_flt["H index"] = [0 if np.isnan(mtid) else get_h_score(int(mtid)) for mtid in staff_flt["MTMT"]]


,MTMT,pubCount,qScore,qnScore,ifCount,ifScore,relifScore,ifNormScore,relifNormScore,citations,hIndex,firstPub,lastPub
0,10012522,19,7.587500,5.739092,8,9.012,6.887500,6.284685,5.039092,76,5,1998,2020
1,10041405,9,1.950000,5.935340,3,0.490,1.200000,1.934906,5.185340,4,1,1982,2009
2,10011729,29,14.387283,10.581248,24,37.300,14.253950,27.004655,10.447914,216,8,2002,2021
3,10042381,13,1.633333,1.633333,0,0.000,0.000000,0.000000,0.000000,1,1,2008,2017
4,10011669,23,5.360267,2.617591,6,12.839,4.210267,4.332244,1.467591,49,4,1985,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,10019824,19,8.545417,6.720328,14,14.789,7.895417,11.317157,6.070328,179,5,2006,2022
115,10019824,19,8.545417,6.720328,14,14.789,7.895417,11.317157,6.070328,179,5,2006,2022
116,10042219,25,18.972517,9.137706,20,43.038,18.905850,19.959357,9.071039,44,4,2015,2023
117,10015273,25,16.528967,10.011277,24,39.862,16.395633,24.277772,9.877943,246,9,2002,2022


# journals

# with affiliation

In [3]:
esi_cats = pd.read_csv("esi-master-journal-list-8-2022.csv")
percentiles = pd.read_csv("baselinecitationpercentiles2022.csv")
percentiles.set_index(keys=["category","year"], inplace=True)

def get_esi_category(jour):
    with open(f'journals/{jour}.json') as fj:
        data = json.load(fj)["content"]
        cat = []
        if "abbreviated" in data:
            cat = esi_cats[esi_cats["Title29"]==data["abbreviated"]]["Category name"]
            if len(cat) == 0:
                cat = esi_cats[esi_cats["Title20"]==data["abbreviated"]]["Category name"]
    return cat.iloc[0]
def check_wos(pub):
    if "journal" in pub and "sciIndexed" in pub["journal"] and pub["journal"]["sciIndexed"]:
        return True
    if "identifiers" in pub:
        for i in pub["identifiers"]:
            if i["source"]["label"] == "WoS" or i["source"]["label"] == "Wos-CSCD (Chinese)":
                return True
    return False
# a függvény visszaadja a folyóirat rangját (D1, Q1, Q2, ...)
def getrating(pub):
    if "ratings" not in pub:
        return ""
    for r in pub["ratings"]:
        if r["otype"] == "SjrRating" and "ranking" in r:
            return r["ranking"]
    return ""

for x, a in tqdm(staff_flt.iterrows()):
    if not np.isnan(a["MTMT"]):
        mtid = int(a["MTMT"])
        if os.path.exists(f'3_3/{mtid}.json'):
            with open(f'3_3/{mtid}.json') as f:
                try:
                    js = json.load(f)
                    for pub in js:      
                        if "journal" in pub and pub["publishedYear"] >= 2012 and check_wos(pub):
                            jour = pub["journal"]["link"].split("/")[-1]
                            if not os.path.exists(f'journals/{jour}.json'):
                                print("Új folyóirat: ", pub["journal"]["label"])
                                os.system(f'wget -O journals/{jour}.json "https://m2.mtmt.hu/api/journal/{jour}"')
                except:
                    pass
        else:
            print("Hiányzó szerző!", mtid)
journal_categories = {}
for x, a in tqdm(staff_flt.iterrows()):
    if not np.isnan(a["MTMT"]):
        mtid = int(a["MTMT"])
        if os.path.exists(f'3_3/{mtid}.json'):
            with open(f'3_3/{mtid}.json') as f:
                try:
                    js = json.load(f)
                    for pub in js:      
                        if "journal" in pub and pub["publishedYear"] >= 2012 and check_wos(pub):
                            jour = pub["journal"]["link"].split("/")[-1]
                            journal_categories[jour] = get_esi_category(jour).upper()
                except:
                    pass
max_year = 2023
pub_sum = {"tsz": [], "év": [], "szerző": [], "cím": [], "mtmt": [], "idézettség": [], "0.01%": [], "0.10%": [], "1.00%": [], "10.00%": [], "20.00%": [], "50.00%": []}
for x, a in tqdm(staff_flt.iterrows()):
    if not np.isnan(a["MTMT"]):
        mtid = int(a["MTMT"])
        if os.path.exists(f'3_3/{mtid}.json'):
            with open(f'3_3/{mtid}.json') as f:
                try:
                    js = json.load(f)
                    for pub in js:     
                        if "journal" in pub and "publishedYear" in pub and pub["publishedYear"] >= 2012 and pub["publishedYear"] <= max_year and check_wos(pub):
                            jour = pub["journal"]["link"].split("/")[-1]
                            if jour in journal_categories.keys() and "citationCount" in pub:
                                try:
                                    base = percentiles.loc[(journal_categories[jour], pub["publishedYear"]), :]
                                except KeyError:
                                    print(pub["journal"]["label"], jour, (journal_categories[jour], pub["publishedYear"]))
                                pub_sum["tsz"].append(a["Tanszék"])
                                pub_sum["év"].append(pub["publishedYear"])
                                pub_sum["szerző"].append(a["Név"])
                                pub_sum["cím"].append(pub["title"])
                                pub_sum["idézettség"].append(pub["citationCount"])
                                pub_sum["mtmt"].append(pub["mtid"])
                                r = (pub["citationCount"] >= base).astype(np.int32)
                                pub_sum["0.01%"].append(r["0.01%"])
                                pub_sum["0.10%"].append(r["0.10%"])
                                pub_sum["1.00%"].append(r["1.00%"])
                                pub_sum["10.00%"].append(r["10.00%"])
                                pub_sum["20.00%"].append(r["20.00%"])
                                pub_sum["50.00%"].append(r["50.00%"])
                except:
                    print(sys.exc_info()[0])
                    pass
pubs = pd.DataFrame(pub_sum).drop_duplicates("mtmt")

119it [00:01, 65.90it/s]
119it [00:02, 47.10it/s]
25it [00:00, 79.46it/s]

LETTERS IN MATHEMATICAL PHYSICS 0377-9017 1573-0530 3035 ('PHYSICS', 2023)
APPLIED MATHEMATICS AND COMPUTATION 0096-3003 1873-5649 454 ('MATHEMATICS', 2023)


58it [00:00, 70.91it/s]

EUROPEAN JOURNAL OF COMBINATORICS 0195-6698 1095-9971 1417130 ('MATHEMATICS', 2023)


73it [00:01, 49.12it/s]

LETTERS IN MATHEMATICAL PHYSICS 0377-9017 1573-0530 3035 ('PHYSICS', 2023)


92it [00:01, 56.01it/s]

JOURNAL OF MATHEMATICAL ANALYSIS AND APPLICATIONS 0022-247X 1096-0813 2634 ('MATHEMATICS', 2023)
EUROPEAN JOURNAL OF COMBINATORICS 0195-6698 1095-9971 1417130 ('MATHEMATICS', 2023)


119it [00:01, 61.10it/s]

JOURNAL OF THE AMERICAN MATHEMATICAL SOCIETY 0894-0347 1088-6834 13001 ('MATHEMATICS', 2023)


In [4]:
pubs

,tsz,év,szerző,cím,mtmt,idézettség,0.01%,0.10%,1.00%,10.00%,20.00%,50.00%
0,Analízis,2018,Andai Attila,Volume of the space of qubit-qubit channels an...,30388770,7,0,0,0,0,0,1
1,Analízis,2017,Andai Attila,Invariance of separability probability over re...,3254502,10,0,0,0,0,0,1
2,Analízis,2016,Andai Attila,Refinement of Robertson-type uncertainty princ...,3127639,1,0,0,0,0,0,0
3,Sztochasztika,2020,Bálint Péter,"Marked Length Spectrum, Homoclinic Orbits and ...",31904067,3,0,0,0,0,0,0
4,Sztochasztika,2019,Bálint Péter,"Polynomial Decay of Correlations for Flows, In...",30946012,11,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
558,Analízis,2013,Weiner Mihály,"Representations of Conformal Nets, Universal C...",2201427,16,0,0,0,0,0,1
559,Analízis,2013,Weiner Mihály,On the snap-back behavior of a self-deploying ...,2520949,16,0,0,0,0,0,1
560,Analízis,2013,Weiner Mihály,A gap for the maximum number of mutually unbia...,1888884,30,0,0,0,1,1,1
561,Analízis,2012,Weiner Mihály,Thermal States in Conformal QFT. II,2201428,15,0,0,0,0,0,1


In [6]:
pubs[pubs["1.00%"]>0]

,tsz,év,szerző,cím,mtmt,idézettség,0.01%,0.10%,1.00%,10.00%,20.00%,50.00%
28,Sztochasztika,2019,Bárány Balázs,Hausdorff dimension of planar self-affine sets...,30603311,45,0,0,1,1,1,1
157,Analízis,2022,Karátson János,Quasi-Newton variable preconditioning for nonl...,32034148,7,0,0,1,1,1,1
289,Sztochasztika,2022,Molontay Roland,EASY-APP: An artificial intelligence model and...,32865751,9,0,0,1,1,1,1
509,Sztochasztika,2015,Vető Bálint,Height Fluctuations for the Stationary KPZ Equ...,2995258,101,0,0,1,1,1,1
552,Analízis,2018,Weiner Mihály,From vertex operator algebras to conformal net...,3327040,61,0,0,1,1,1,1


In [29]:
pubs.to_csv("percentille_affiliation.csv", index=False, float_format='%.4g')

# no aff

In [3]:
esi_cats = pd.read_csv("esi-master-journal-list-8-2022.csv")
percentiles = pd.read_csv("baselinecitationpercentiles2022.csv")
percentiles.set_index(keys=["category","year"], inplace=True)

def get_esi_category(jour):
    with open(f'journals/{jour}.json') as fj:
        data = json.load(fj)["content"]
        cat = []
        if "abbreviated" in data:
            cat = esi_cats[esi_cats["Title29"]==data["abbreviated"]]["Category name"]
            if len(cat) == 0:
                cat = esi_cats[esi_cats["Title20"]==data["abbreviated"]]["Category name"]
    return cat.iloc[0]
def check_wos(pub):
    if "journal" in pub and "sciIndexed" in pub["journal"] and pub["journal"]["sciIndexed"]:
        return True
    if "identifiers" in pub:
        for i in pub["identifiers"]:
            if i["source"]["label"] == "WoS" or i["source"]["label"] == "Wos-CSCD (Chinese)":
                return True
    return False
# a függvény visszaadja a folyóirat rangját (D1, Q1, Q2, ...)
def getrating(pub):
    if "ratings" not in pub:
        return ""
    for r in pub["ratings"]:
        if r["otype"] == "SjrRating" and "ranking" in r:
            return r["ranking"]
    return ""

for x, a in tqdm(staff_flt.iterrows()):
    if not np.isnan(a["MTMT"]):
        mtid = int(a["MTMT"])
        if os.path.exists(f'3_1/{mtid}.json'):
            with open(f'3_1/{mtid}.json') as f:
                try:
                    js = json.load(f)
                    for pub in js:      
                        if "journal" in pub and pub["publishedYear"] >= 2012 and check_wos(pub):
                            jour = pub["journal"]["link"].split("/")[-1]
                            if not os.path.exists(f'journals/{jour}.json'):
                                print("Új folyóirat: ", pub["journal"]["label"])
                                os.system(f'wget -O journals/{jour}.json "https://m2.mtmt.hu/api/journal/{jour}"')
                except:
                    pass
        else:
            print("Hiányzó szerző!", mtid)
journal_categories = {}
for x, a in tqdm(staff_flt.iterrows()):
    if not np.isnan(a["MTMT"]):
        mtid = int(a["MTMT"])
        if os.path.exists(f'3_1/{mtid}.json'):
            with open(f'3_1/{mtid}.json') as f:
                try:
                    js = json.load(f)
                    for pub in js:      
                        if "journal" in pub and pub["publishedYear"] >= 2012 and check_wos(pub):
                            jour = pub["journal"]["link"].split("/")[-1]
                            journal_categories[jour] = get_esi_category(jour).upper()
                except:
                    pass
max_year = 2023
pub_sum = {"tsz": [], "év": [], "szerző": [], "cím": [], "mtmt": [], "idézettség": [], "0.01%": [], "0.10%": [], "1.00%": [], "10.00%": [], "20.00%": [], "50.00%": []}
for x, a in tqdm(staff_flt.iterrows()):
    if not np.isnan(a["MTMT"]):
        mtid = int(a["MTMT"])
        if os.path.exists(f'3_1/{mtid}.json'):
            with open(f'3_1/{mtid}.json') as f:
                try:
                    js = json.load(f)
                    for pub in js:     
                        if "journal" in pub and "publishedYear" in pub and pub["publishedYear"] >= 2012 and pub["publishedYear"] <= max_year and check_wos(pub):
                            jour = pub["journal"]["link"].split("/")[-1]
                            if jour in journal_categories.keys() and "citationCount" in pub:
                                try:
                                    base = percentiles.loc[(journal_categories[jour], pub["publishedYear"]), :]
                                except KeyError:
                                    print(pub["journal"]["label"], jour, (journal_categories[jour], pub["publishedYear"]))
                                pub_sum["tsz"].append(a["Tanszék"])
                                pub_sum["év"].append(pub["publishedYear"])
                                pub_sum["szerző"].append(a["Név"])
                                pub_sum["cím"].append(pub["title"])
                                pub_sum["idézettség"].append(pub["citationCount"])
                                pub_sum["mtmt"].append(pub["mtid"])
                                r = (pub["citationCount"] >= base).astype(np.int32)
                                pub_sum["0.01%"].append(r["0.01%"])
                                pub_sum["0.10%"].append(r["0.10%"])
                                pub_sum["1.00%"].append(r["1.00%"])
                                pub_sum["10.00%"].append(r["10.00%"])
                                pub_sum["20.00%"].append(r["20.00%"])
                                pub_sum["50.00%"].append(r["50.00%"])
                except:
                    print(sys.exc_info()[0])
                    pass
pubs = pd.DataFrame(pub_sum).drop_duplicates("mtmt")

119it [00:05, 20.03it/s]
119it [00:06, 17.38it/s]
6it [00:00, 48.41it/s]

LETTERS IN MATHEMATICAL PHYSICS 0377-9017 1573-0530 3035 ('PHYSICS', 2023)


28it [00:02, 13.79it/s]

MATHEMATICAL MODELLING AND ANALYSIS 1392-6292 1648-3510 10011718 ('MATHEMATICS', 2023)
APPLIED MATHEMATICS AND COMPUTATION 0096-3003 1873-5649 454 ('MATHEMATICS', 2023)


39it [00:02, 18.53it/s]

...

JOURNAL OF MATHEMATICAL ANALYSIS AND APPLICATIONS 0022-247X 1096-0813 2634 ('MATHEMATICS', 2023)


89it [00:05, 25.78it/s]

EUROPEAN JOURNAL OF COMBINATORICS 0195-6698 1095-9971 1417130 ('MATHEMATICS', 2023)


119it [00:06, 19.74it/s]

JOURNAL OF THE AMERICAN MATHEMATICAL SOCIETY 0894-0347 1088-6834 13001 ('MATHEMATICS', 2023)


In [31]:
pubs[pubs["1.00%"]>0]

,tsz,év,szerző,cím,mtmt,idézettség,0.01%,0.10%,1.00%,10.00%,20.00%,50.00%
31,Sztochasztika,2019,Bárány Balázs,Hausdorff dimension of planar self-affine sets...,30603311,45,0,0,1,1,1,1
174,Algebra,2016,Hegedűs Pál,On the Discrete Logarithmic Minkowski Problem,3121562,69,0,0,1,1,1,1
213,Analízis,2022,Karátson János,Quasi-Newton variable preconditioning for nonl...,32034148,7,0,0,1,1,1,1
308,Differenciálegyenletek,2018,Kovács Mihály,Boundary conditions for fractional diffusion,3348787,45,0,0,1,1,1,1
433,Sztochasztika,2022,Molontay Roland,EASY-APP: An artificial intelligence model and...,32865751,9,0,0,1,1,1,1
682,Sztochasztika,2015,Vető Bálint,Height Fluctuations for the Stationary KPZ Equ...,2995258,101,0,0,1,1,1,1
727,Analízis,2018,Weiner Mihály,From vertex operator algebras to conformal net...,3327040,61,0,0,1,1,1,1


In [32]:
pubs.to_csv("percentille_no_affiliation.csv", index=False, float_format='%.4g')

# citation publication

# no aff

In [15]:
def load_pub_for_author(mtid):
    tries = 1
    page = 1
    while True:
        os.system(f'wget -O tmp.json "https://m2.mtmt.hu/api/publication?cond=authors.mtid;eq;{mtid}&page={page}&format=json" >wget.out 2>wget.err')
        with open("tmp.json", "r") as f:
            try:
                dta = json.load(f)
                if not os.path.exists(f"journals3/{mtid}{page}.json"):
                    json.dump(dta, open(f"journals3/{mtid}{page}.json", "w", encoding='utf8'), indent=4, ensure_ascii=False)
                if not dta["paging"]["last"]:
                    page += 1
                else:
                    break
            except:
                tries += 1
                print("Error, retrying")
                if tries > 30:
                    return []
staff_flt=pd.read_csv('people_flt.csv')
for mtid in tqdm(staff_flt["MTMT"]):
    load_pub_for_author(mtid)
df=pd.DataFrame(columns =['publishedYear','name','authors','independentCitingPubCount','types'])
for files in os.listdir('journals3/'):
    dta = json.load(open(f'journals3/{files}'))
    for x in dta["content"]:
        auth=""
        for y in x["authorships"]:
            if "author" in y:
                auth=auth+str(y["author"]["mtid"])+","
        a=pd.Series({'publishedYear':x["publishedYear"],'name':x["title"],'authors':auth,'independentCitingPubCount':x["independentCitationCount"],'types':x["otype"]})
        df=pd.concat([df, a.to_frame().T], ignore_index=True)
df=df.drop_duplicates(subset=['name'])
df = df[ (df['types'] != 'Thesis') & (df['types'] != 'Achievement') ]
df

100%|█████████████████████████████████████████| 124/124 [16:44<00:00,  8.10s/it]


,publishedYear,name,authors,independentCitingPubCount,types
0,2000,Három halk visszhang,"10000407,",1,JournalArticle
1,2000,On a conjecture of Kemnitz,"10000407,",35,JournalArticle
2,1999,Norm-graphs: variations and applications,"10000407,",97,JournalArticle
3,1999,On plus-minus 1-representations of integers,"10000952,10000413,10000407,",5,JournalArticle
4,1999,Computations in associative and Lie algebras,"10000741,10000407,",6,BookChapter
...,...,...,...,...,...
5011,2003,Operator Splitting and Global Error Analysis,"10039378,10004964,10011854,",0,JournalArticle
5012,2003,Parallel computations with large-scale air pol...,"10004964,",0,JournalArticle
5014,2003,Galerkin approximations for the linear parabol...,"10004964,",0,JournalArticle
5017,2002,Sharpening the estimate of the stability const...,"10004964,",11,JournalArticle


In [18]:
x=df['authors'].str.split(',',expand=True)
x=x.drop([32], axis=1)
x=df.join(x)
staff_flt["MTMT"] = staff_flt["MTMT"].apply(str)
for i in range(32):
    mask = x[i].isin(staff_flt["MTMT"])
    x[i] = x[i].where(mask, None)
for i in range(32):
    y=staff_flt
    pubs=y.rename(columns={"MTMT": i})
    x=pd.merge(x, pubs[[i , "Név"]], how='left',  on=[i])
    x=x.drop([i], axis=1)
    x=x.rename(columns={"Név": i})
x=x.drop(["authors"], axis=1)
x['authors'] = x[x.columns[4:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)
x=x[['publishedYear','name','authors','independentCitingPubCount','types']]

In [19]:
x.to_csv("independentCitingPubCountnoaffiliation.csv", index=False, float_format='%.4gd')

#  aff

In [10]:
def load_pub_for_author(mtid):
    tries = 1
    page = 1
    while True:
        os.system(f'wget -O tmp.json "https://m2.mtmt.hu/api/publication?cond=authors.mtid;eq;{mtid}&cond=institutes;inia;10856&&page={page}&format=json" >wget.out 2>wget.err')
        with open("tmp.json", "r") as f:
            try:
                dta = json.load(f)
                if not os.path.exists(f"journals4/{mtid}{page}.json"):
                    json.dump(dta, open(f"journals4/{mtid}{page}.json", "w", encoding='utf8'), indent=4, ensure_ascii=False)
                if not dta["paging"]["last"]:
                    page += 1
                else:
                    break
            except:
                tries += 1    
                print("Error, retrying")
                if tries > 30:
                    return []
staff_flt=pd.read_csv('people_flt.csv')
for mtid in tqdm(staff_flt["MTMT"]):
    load_pub_for_author(mtid)
df=pd.DataFrame(columns =['publishedYear','name','authors','independentCitingPubCount','types'])
for files in os.listdir('journals4/'):
    dta = json.load(open(f'journals4/{files}'))
    for x in dta["content"]:
        auth=""
        for y in x["authorships"]:
            if "author" in y:
                auth=auth+str(y["author"]["mtid"])+","
        a=pd.Series({'publishedYear':x["publishedYear"],'name':x["title"],'authors':auth,'independentCitingPubCount':x["independentCitationCount"],'types':x["otype"]})
        df=pd.concat([df, a.to_frame().T], ignore_index=True)
df=df.drop_duplicates(subset=['name'])
df = df[ (df['types'] != 'Thesis') & (df['types'] != 'Achievement') ]
df

100%|█████████████████████████████████████████| 124/124 [37:26<00:00, 18.12s/it]


,publishedYear,name,authors,independentCitingPubCount,types
0,2020,SPECTRAL CHARACTERIZATION OF JORDAN-SEGAL ISOM...,"10000799,",1,JournalArticle
1,2019,Characterizations of certain means of positive...,"10000799,",1,JournalArticle
2,2019,On 2-local *-automorphisms and 2-local isometr...,"10000799,",6,JournalArticle
3,2019,Quantum Rényi relative entropies on density sp...,"10000799,",0,JournalArticle
4,2018,Busch-Gudder metric on the Cone of Positive Se...,"10000799,",1,JournalArticle
...,...,...,...,...,...
2489,1993,Matematikai feladatgyűjtemény III.,"10041403,",0,Book
2492,1990,On generalized cosquares,"10041403,",0,JournalArticle
2493,1986,Simultaneous triangularization of projector ma...,"10041403,",4,JournalArticle
2494,1981,On a groupoid of n-tuples of numbers,"10041403,",0,JournalArticle


In [12]:
x=df['authors'].str.split(',',expand=True)
x=x.drop([32], axis=1)
x=df.join(x)
staff_flt["MTMT"] = staff_flt["MTMT"].apply(str)
for i in range(32):
    mask = x[i].isin(staff_flt["MTMT"])
    x[i] = x[i].where(mask, None)
for i in range(32):
    y=staff_flt
    pubs=y.rename(columns={"MTMT": i})
    x=pd.merge(x, pubs[[i , "Név"]], how='left',  on=[i])
    x=x.drop([i], axis=1)
    x=x.rename(columns={"Név": i})
x=x.drop(["authors"], axis=1)
x['authors'] = x[x.columns[4:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)
x=x[['publishedYear','name','authors','independentCitingPubCount','types']]

In [14]:
x.to_csv("independentCitingPubCountaffiliation.csv", index=False, float_format='%.4gd')

In [13]:
x

,publishedYear,name,authors,independentCitingPubCount,types
0,2020,SPECTRAL CHARACTERIZATION OF JORDAN-SEGAL ISOM...,Molnár Lajos Gábor,1,JournalArticle
1,2019,Characterizations of certain means of positive...,Molnár Lajos Gábor,1,JournalArticle
2,2019,On 2-local *-automorphisms and 2-local isometr...,Molnár Lajos Gábor,6,JournalArticle
3,2019,Quantum Rényi relative entropies on density sp...,Molnár Lajos Gábor,0,JournalArticle
4,2018,Busch-Gudder metric on the Cone of Positive Se...,Molnár Lajos Gábor,1,JournalArticle
...,...,...,...,...,...
2052,1993,Matematikai feladatgyűjtemény III.,Szép Gabriella,0,Book
2053,1990,On generalized cosquares,Szép Gabriella,0,JournalArticle
2054,1986,Simultaneous triangularization of projector ma...,Szép Gabriella,4,JournalArticle
2055,1981,On a groupoid of n-tuples of numbers,Szép Gabriella,0,JournalArticle
